In [1]:
from typing import List

import pennylane as qml
from qiskit_ibm_provider import IBMProvider
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global

In [2]:
SIMULATION_MODE = True    # running from local or real hardware

In [3]:
from mitiq.benchmarks import generate_ghz_circuit
from mitiq.zne.scaling import insert_id_layers, fold_global

demo = generate_ghz_circuit(3)


print("-----ORIGINAL-----")
print(demo)
print("\n-----------------FOLDING------------------")
for f in range(2,8):
    print(f)
    print(fold_global(demo, f))
    print()

-----ORIGINAL-----
0: ───H───@───────
          │
1: ───────X───@───
              │
2: ───────────X───

-----------------FOLDING------------------
2
0: ───H───@───────────@───@───────
          │           │   │
1: ───────X───@───@───X───X───@───
              │   │           │
2: ───────────X───X───────────X───

3
0: ───H───@───────────@───H───H───@───────
          │           │           │
1: ───────X───@───@───X───────────X───@───
              │   │                   │
2: ───────────X───X───────────────────X───

4
0: ───H───@───────────@───H───H───@───────────@───@───────
          │           │           │           │   │
1: ───────X───@───@───X───────────X───@───@───X───X───@───
              │   │                   │   │           │
2: ───────────X───X───────────────────X───X───────────X───

5
0: ───H───@───────────@───H───H───@───────────@───H───H───@───────
          │           │           │           │           │
1: ───────X───@───@───X───────────X───@───@───X───────────X

##  Build a simple noise model with depolarizing noise

In [4]:
if SIMULATION_MODE:
    noise_strength = 0.05
    dev_noise_free = qml.device("default.mixed", wires=2)
    dev = qml.transforms.insert(
        dev_noise_free,
        qml.DepolarizingChannel,
        noise_strength
    )
elif IBMProvider.saved_accounts() and USE_REAL_HARDWARE:
    provider = IBMProvider()
    dev = qml.device(
        "qiskit.ibmq",
        wires=2,
        backend="ibmq_qasm_simulator",
        provider=provider
    )    

In [5]:
def circuit():
    """
    @A circuit preparing a Bell state
    """
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

Zero-noise extrapolation (ZNE) is a noise mitigation technique. It works by intentionally scaling the noise of a quantum circuit to then extrapolate the zero-noise limit of an observable of interest. In this task, you will build a simple ZNE function from scratch:

- [x] Build a simple noise model with depolarizing noise 
- [x] Create different circuits to test your noise models and choose the observable to measure 
- [ ] Apply the unitary folding method. 
- [ ] Apply the extrapolation method to get the zero-noise limit. Different extrapolation methods achieve different results, such as Linear, polynomial, and exponential.
- [x] Compare mitigated and unmitigated results 
- [ ] Bonus: Run your ZNE function in real quantum hardware through the IBM Quantum Service

Check the Mitiq documentation for references. You are not allowed to use the functions from Mitiq or any other frameworks where ZNE is already implemented. 


## todo Applying unitary folding method

### Implement

In [27]:
def unitary_fold(circuit, scale_factor: int):
    # original ops
    circuit()
    original_ops = circuit.tape.expand().operations
    ops = circuit.tape.expand().copy(copy_operations=True).operations
    n, s = divmod(scale_factor - 1, 2)
    
    # take all the gate, append and complex conjugate
    # For the 1st part  (U^H U)**n
    for i in range(n):
        for op in original_ops[::-1]:
            ops.append(qml.adjoint(op))
        for op in original_ops:
            ops.append(op)
    
    # For the 2nd part (L_d^H .. L_s^H) (L_s .. L_d)
    last_layers = original_ops[-s:]
    for op in last_layers[::-1]:
        ops.append(qml.adjoint(op))
    for i in last_layers:
        ops.append(op)

    # Return list of op to create the circuit
    return ops, circuit.tape.measurements

In [101]:
@qml.qnode(dev)
def circuit_from_ops(operations: List, measurements: List):
    for op in operations:
        qml.apply(op)
    return qml.apply(measurements[0])

In [104]:
device_circuit = qml.QNode(circuit, dev)
ops, measurements = unitary_fold(device_circuit, 2)
circuit_from_ops(ops, measurements)

array(0.7082456)

In [105]:
device_circuit()

tensor(0.87111111, requires_grad=True)

In [106]:
circuit_from_ops(ops, measurements)

array(0.7082456)

In [107]:
print(qml.draw(circuit_from_ops)(ops, measurements))

0: ──Rϕ(1.57)──RX(1.57)──Rϕ(1.57)─╭●─╭●──╭●─┤ ╭<Z@Z>
1: ───────────────────────────────╰X─╰X†─╰X─┤ ╰<Z@Z>


### Test the implementation

## todo Linear, polynomial, and exponential extrapolation

## Create a reference ZNE mitigated circuit

In [ ]:
scale_factors = [1,2,3,4,5,6,7,8]

error_mitigated_device_circuit = qml.transforms.mitigate_with_zne(
    device_circuit,
    scale_factors,
    folding = fold_global,    
    extrapolate = RichardsonFactory.extrapolate,
)

## Compare the results between noisy circuit, reference method, and implemented method

In [ ]:
unmitigated = device_circuit()
mitigated = error_mitigated_device_circuit()
print(f"Unmitigated result {unmitigated}")
print(f"Mitigated result   {mitigated}")

In [ ]:
print(qml.draw(error_mitigated_device_circuit)())

As the ideal, desired result is 1.000, the mitigated result performs much better than unmitigated.